## Imports

In [12]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import cv2
from skimage import exposure
import matplotlib.pyplot as plt


In [13]:
# Set the paths to the folders containing the training and validation data
train_path = 'train1'
val_path = 'val1'

# Set the batch size and image dimensions for the input to the model
batch_size = 10
img_size = (224, 224)


## Preprocessing


In [14]:
def preprocess_image(img):
    # # Read the image
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Apply contrast stretching
    p2, p98 = np.percentile(img, (10, 90))
    img = exposure.rescale_intensity(img, in_range=(p2, p98))

    # Normalize the image
    img = (img - np.min(img)) / (np.max(img) - np.min(img))

    # Convert the image to 3 channels
    img = cv2.merge([img, img, img])
    # plt.imshow(img)
    # plt.title('img')
    # plt.show()
    return img


In [15]:
# Create an instance of the ImageDataGenerator class for data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_image,
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

# Create an instance of the ImageDataGenerator class for validation data
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image,
    rescale=1./255,
    horizontal_flip=True
)

# Create generators for the training and validation data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)


Found 111 images belonging to 2 classes.
Found 36 images belonging to 2 classes.


## Models

### pretrained model with all base layers frozen.

In [16]:
# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(
    input_shape=img_size+(3,),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add a new output layer to the model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
output_layer = Dense(2, activation='softmax')(x)
model1 = Model(inputs=base_model.input, outputs=output_layer)

### pretrained model with last 20 base layers unfrozen

In [17]:
# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(
    input_shape=img_size+(3,),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-20:]:
    layer.trainable = True

# Add a new output layer to the model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
output_layer = Dense(2, activation='softmax')(x)
model2 = Model(inputs=base_model.input, outputs=output_layer)

### model fully trained with our dataset

In [18]:
# Load the pre-trained MobileNetV3 model
base_model = MobileNetV2(
    input_shape=img_size+(3,),
    include_top=False,
    weights='imagenet'
)

# Add a new output layer to the model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
output_layer = Dense(2, activation='softmax')(x)
model3 = Model(inputs=base_model.input, outputs=output_layer)

## Compile

In [19]:
# Compile the models with categorical cross-entropy loss and Adam optimizer
model1.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model2.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model3.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Create a ModelCheckpoint callback to save the best weights based on the validation loss
checkpoint = ModelCheckpoint(
    'best_weights.h5',
    monitor='val_loss',
    save_best_only=False,
    save_weights_only=True,
    mode='min',
    verbose=1
)


## Train the models

In [20]:
# Train the model using the generators
model1.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[checkpoint]
)
model2.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[checkpoint]
)
model3.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[checkpoint]
)


Epoch 1/20
12/12 [==============================] - ETA: 0s - loss: 1.0476 - accuracy: 0.4865
Epoch 1: saving model to best_weights.h5
12/12 [==============================] - 9s 457ms/step - loss: 1.0476 - accuracy: 0.4865 - val_loss: 0.7903 - val_accuracy: 0.6389
Epoch 2/20
12/12 [==============================] - ETA: 0s - loss: 0.9943 - accuracy: 0.5135
Epoch 2: saving model to best_weights.h5
12/12 [==============================] - 4s 373ms/step - loss: 0.9943 - accuracy: 0.5135 - val_loss: 0.8389 - val_accuracy: 0.3611
Epoch 3/20
12/12 [==============================] - ETA: 0s - loss: 0.9510 - accuracy: 0.4595
Epoch 3: saving model to best_weights.h5
12/12 [==============================] - 4s 347ms/step - loss: 0.9510 - accuracy: 0.4595 - val_loss: 0.7494 - val_accuracy: 0.3611
Epoch 4/20
12/12 [==============================] - ETA: 0s - loss: 0.8121 - accuracy: 0.5676
Epoch 4: saving model to best_weights.h5
12/12 [==============================] - 4s 360ms/step - loss: 0.81

In [21]:
# model1.evaluate(val_generator)
# model2.evaluate(val_generator)
# model3.evaluate(val_generator)

## Results

In [25]:
# Define the test data generator and load the test set
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image,
    rescale=1./255)
test_set = test_datagen.flow_from_directory(
    "val1",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Get the true labels of the test set
true_labels = test_set.classes

# Make predictions on the test set
pred_probs_1 = model1.predict(test_set, verbose=1)
pred_probs_2 = model2.predict(test_set, verbose=1)
pred_probs_3 = model3.predict(test_set, verbose=1)
# print(pred_probs_1)
pred_labels_1 = np.argmax(pred_probs_1, axis=1)
pred_labels_2 = np.argmax(pred_probs_2, axis=1)
pred_labels_3 = np.argmax(pred_probs_3, axis=1)
print(true_labels)
print(pred_labels_1)
print(pred_labels_2)
print(pred_labels_3)

# Create the confusion matrix
cm1 = confusion_matrix(true_labels, pred_labels_1)
cm2 = confusion_matrix(true_labels, pred_labels_2)
cm3 = confusion_matrix(true_labels, pred_labels_3)
print("model1: All base layers frozen.")
print(cm1)
print(classification_report(true_labels, pred_labels_1, target_names=['Reccurant', 'Non-reccurant']))
print("model2: All base layers frozen but last 20 unfrozen.")
print(cm2)
print(classification_report(true_labels, pred_labels_2, target_names=['Reccurant', 'Non-reccurant']))
print("model3: Training all layers with our dataset.")
print(cm3)
print(classification_report(true_labels, pred_labels_3, target_names=['Reccurant', 'Non-reccurant']))

# # Calculate the accuracy, precision, recall, and f1 score
# accuracy = accuracy_score(true_labels, pred_labels)
# precision = precision_score(true_labels, pred_labels)
# recall = recall_score(true_labels, pred_labels)
# f1 = f1_score(true_labels, pred_labels)


# # Print the results
# print("Confusion Matrix:\n", cm)
# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)


Found 36 images belonging to 2 classes.
4/4 [==============================] - 1s 216ms/step
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
model1: All base layers frozen.
[[23  0]
 [13  0]]
               precision    recall  f1-score   support

    Reccurant       0.64      1.00      0.78        23
Non-reccurant       0.00      0.00      0.00        13

     accuracy                           0.64        36
    macro avg       0.32      0.50      0.39        36
 weighted avg       0.41      0.64      0.50        36

model2: All base layers frozen but last 20 unfrozen.
[[23  0]
 [13  0]]
               precision    recall  f1-score   support

    Reccurant       0.64      1.00      0.78        23
Non-reccurant       0.00      0.00      0.00

c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarn

In [23]:
# from tensorflow.keras.optimizers import Adam

# for layer in model.layers[len(model.layers)-7:]:
#     layer.trainable = True

# optimizer = Adam(learning_rate=0.0001)

# model.compile(
#     loss='categorical_crossentropy',
#     optimizer=optimizer,
#     metrics=['accuracy', 'Precision', 'Recall']
# )
